# Fingerprint-Based Online Attendance System Using ESP32
This code integrates an ESP32 with an Adafruit fingerprint sensor to authenticate users and mark attendance by sending fingerprint IDs to a Google Sheets-based online attendance system via WiFi. It verifies fingerprints, retrieves matching IDs, and logs attendance through an HTTPS request.

# Steps:

## 1. Including Required Libraries


In [ ]:
#include <Adafruit_Fingerprint.h>
#include <WiFi.h>
#include <HTTPClient.h>
#include <WiFiClient.h>
#include <WiFiClientSecure.h>

- Adafruit_Fingerprint.h: Library for interfacing with the fingerprint sensor.
- WiFi.h: Enables ESP32 to connect to a WiFi network.
- HTTPClient.h: Allows sending HTTP requests.
- WiFiClient.h & WiFiClientSecure.h: Used for secure HTTPS communication.

## 2. Configuring Serial Communication

In [ ]:
#if (defined(__AVR__) || defined(ESP8266)) && !defined(__AVR_ATmega2560__)
SoftwareSerial mySerial(2, 3);
#else
#define mySerial Serial2
#endif
#if (defined(__AVR__) || defined(ESP8266)) && !defined(__AVR_ATmega2560__)
SoftwareSerial mySerial(2, 3);
#else
#define mySerial Serial2
#endif

- Uses SoftwareSerial for boards without multiple hardware serial ports.
- Uses Serial2 for boards with multiple hardware serial ports.

## 3. Initializing Fingerprint Sensor and WiFi Credentials

In [ ]:
Adafruit_Fingerprint finger = Adafruit_Fingerprint(&mySerial);

const char* ssid = "atNet";        // WiFi SSID
const char* password = "12345678"; // WiFi Password

const String sheet_url = "https://script.google.com/macros/s/AKfycbzb8K9t87pFqnAZILGvnAmnX-vt8c-EJxGMQjcdwYSBddPnNL4RuLwRGigMFXfpnQ7J/exec";

String req;


- Initializes the fingerprint sensor object.
- Stores WiFi credentials for ESP32 to connect.
- Defines the Google Apps Script URL to send attendance data.

## 4. Setting up WiFi and Fingerprint Sensor

In [ ]:
void setup() {
  Serial.begin(115200);

  WiFi.mode(WIFI_STA);  // Set WiFi mode to Station
  WiFi.begin(ssid, password);
  Serial.println("\nConnecting");

  while (WiFi.status() != WL_CONNECTED) {
    Serial.print(".");
    delay(100);
  }

  Serial.println("\nConnected to the WiFi network");
  Serial.print("Local ESP32 IP: ");
  Serial.println(WiFi.localIP());

  while (!Serial);  
  delay(100);
  Serial.println("\n\nAdafruit finger detect test");

  finger.begin(57600);
  delay(5);

  if (finger.verifyPassword()) {
    Serial.println("Found fingerprint sensor!");
  } else {
    Serial.println("Did not find fingerprint sensor :(");
    while (1) { delay(1); }
  }

  Serial.println(F("Reading sensor parameters"));
  finger.getParameters();
  Serial.print(F("Status: 0x"));
  Serial.println(finger.status_reg, HEX);
  Serial.print(F("Sys ID: 0x"));
  Serial.println(finger.system_id, HEX);
  Serial.print(F("Capacity: "));
  Serial.println(finger.capacity);
  Serial.print(F("Security level: "));
  Serial.println(finger.security_level);
  Serial.print(F("Device address: "));
  Serial.println(finger.device_addr, HEX);
  Serial.print(F("Packet len: "));
  Serial.println(finger.packet_len);
  Serial.print(F("Baud rate: "));
  Serial.println(finger.baud_rate);

  finger.getTemplateCount();

  if (finger.templateCount == 0) {
    Serial.print("Sensor doesn't contain any fingerprint data. Please run the 'enroll' example.");
  } else {
    Serial.println("Waiting for valid finger...");
    Serial.print("Sensor contains ");
    Serial.print(finger.templateCount);
    Serial.println(" templates");
  }
}


- Connects to WiFi and displays the local IP address.
- Initializes the fingerprint sensor and reads its parameters.
- Displays the number of stored fingerprints.

## 5. Main Loop for Fingerprint Recognition


In [ ]:
void loop() {
  getFingerprintID();
  delay(50);
}


- Calls getFingerprintID() continuously to detect and verify fingerprints.

## 6. Capturing and Processing Fingerprint

In [ ]:
uint8_t getFingerprintID() {
  uint8_t p = finger.getImage();
  switch (p) {
    case FINGERPRINT_OK:
      Serial.println("Image taken");
      break;
    case FINGERPRINT_NOFINGER:
      Serial.println("No finger detected");
      return p;
    case FINGERPRINT_PACKETRECIEVEERR:
      Serial.println("Communication error");
      return p;
    case FINGERPRINT_IMAGEFAIL:
      Serial.println("Imaging error");
      return p;
    default:
      Serial.println("Unknown error");
      return p;
  }

  p = finger.image2Tz();
  switch (p) {
    case FINGERPRINT_OK:
      Serial.println("Image converted");
      break;
    case FINGERPRINT_IMAGEMESS:
      Serial.println("Image too messy");
      return p;
    case FINGERPRINT_PACKETRECIEVEERR:
      Serial.println("Communication error");
      return p;
    case FINGERPRINT_FEATUREFAIL:
      Serial.println("Could not find fingerprint features");
      return p;
    case FINGERPRINT_INVALIDIMAGE:
      Serial.println("Could not find fingerprint features");
      return p;
    default:
      Serial.println("Unknown error");
      return p;
  }

  p = finger.fingerSearch();
  if (p == FINGERPRINT_OK) {
    Serial.println("Found a print match!");
  } else if (p == FINGERPRINT_PACKETRECIEVEERR) {
    Serial.println("Communication error");
    return p;
  } else if (p == FINGERPRINT_NOTFOUND) {
    Serial.println("Did not find a match");
    return p;
  } else {
    Serial.println("Unknown error");
    return p;
  }

  uint8_t finger_id = finger.fingerID;
  Serial.print("Found ID #");
  Serial.print(finger_id);
  Serial.print(" with confidence of ");
  Serial.println(finger.confidence);

  markAttendance(finger_id);

  delay(5000);
  return finger.fingerID;
}


- Captures an image of the fingerprint.
- Converts the image into a digital template.
- Searches the database for a match and returns the ID.
- If a match is found, it marks attendance by calling markAttendance().

## 7. Simplified Fingerprint Matching

In [ ]:
int getFingerprintIDez() {
  uint8_t p = finger.getImage();
  if (p != FINGERPRINT_OK) return -1;

  p = finger.image2Tz();
  if (p != FINGERPRINT_OK) return -1;

  p = finger.fingerFastSearch();
  if (p != FINGERPRINT_OK) return -1;

  Serial.print("Found ID #");
  Serial.print(finger.fingerID);
  Serial.print(" with confidence of ");
  Serial.println(finger.confidence);
  return finger.fingerID;
}


## 8. Marking Attendance in Google Sheets

In [ ]:
void markAttendance(uint8_t fingerPrintID) {
  if (WiFi.status() == WL_CONNECTED) {
    
    WiFiClientSecure client;
    client.setInsecure(); // Consider using client.setCACert for better security

    String req = String(sheet_url) + "?id=" + String(fingerPrintID);
    req.trim();
    Serial.println(req);

    HTTPClient https;
    Serial.println(F("[HTTPS] begin..."));

    if (https.begin(client, req)) {
      Serial.println(F("[HTTPS] GET..."));
      int httpCode = https.GET();

      if (httpCode > 0) {
        Serial.printf("[HTTPS] GET... code: %d\n", httpCode);
        if (httpCode == HTTP_CODE_OK) {
          String payload = https.getString();
          Serial.println("Received payload: " + payload);
        }
      } else {
        Serial.printf("[HTTPS] GET... failed, error: %s\n", https.errorToString(httpCode).c_str());
      }
      https.end();
    } else {
      Serial.println(F("[HTTPS] Unable to connect"));
    }
  } else {
    Serial.println(F("WiFi not connected"));
  }
  delay(1000);
}


- Sends fingerprint ID to a Google Apps Script.
- Uses HTTPS request to log attendance in Google Sheets.
- Checks for WiFi connection before sending data.

## Summary
- This ESP32-based system detects fingerprints, matches IDs, and logs attendance in Google Sheets.
- Uses WiFi and HTTPS requests for real-time attendance tracking.
- Handles fingerprint image processing, ID retrieval, and error checking.